In [32]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [38]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "inlet"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
#geo.AddCircle ( (-0.5, 0.5), r=0.2, leftdomain=0, rightdomain=1, bc="cyl2")

mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
# viscosity
nu = 0.001
k = 3
V = VectorH1(mesh,order=k, dirichlet="top|bot|cyl|inlet|out|cyl2")
Q = H1(mesh,order=k-1)
X = V*Q

In [39]:
gfu = GridFunction(X)
velocity = gfu.components[0]
scene = Draw(velocity,mesh,"u")#,sd=3)
#scene2 = Draw(gfu.components[1],mesh,"p")#,sd=3)
#visoptions.scalfunction = "u:0"

# parabolic inflow at bc=1:
uinf = 0.001
uin = CoefficientFunction((uinf,0))
ball = CoefficientFunction((0,0))
init = uin + ball
#gfu.components[0].Set(ball, definedon=mesh.Boundaries("cyl"))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet|top|bot|out"))

#gfu.componets[0] += gfu.components[0].Set(ball, definedon=mesh.Boundaries("cyl"))

# first component of gfu.components bedlongs to v ans has dim=2, [1] is tShe pressure broski
scene.Redraw()
#scene2.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [41]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

#drag
drag_x_test = GridFunction(X)
drag_x_test.components[0].Set(CoefficientFunction((10.0,0)), definedon=mesh.Boundaries("cyl"))

# calc going on
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

# somehow only works the 2nd time
print(InnerProduct(res, drag_x_test.vec))

scene.Redraw()

0.0002197740785547812


In [31]:
# getting the drag in here
drag_x_test = GridFunction(X)
drag_x_test.components[0].Set(CoefficientFunction((-20.0,0)), definedon=mesh.Boundaries("cyl"))
drag_x_vals = []
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
drag_x_vals.append(InnerProduct(res, drag_x_test.vec) )
print(drag_x_vals)

[-0.00040579870906503495]
